In [1]:
import sys
import os

sys.path.append(os.path.dirname(os.getcwd()))

os.chdir("../")

# Normalisation

In [3]:
import xarray as xr
import zarr
import numcodecs
import numpy as np
import glob
import torch
import era5_data_proc
import os

RAW_ERA5_PATH = "/vol/bitbucket/bet20/dataset/era5/global_full"
# RAW_ERA5_PATH = '/work/ec249/ec249/bet20/dataset/era5/global_full'


## Checking MEPS files

In [3]:
import neural_lam.utils

stats = neural_lam.utils.load_dataset_stats("meps_example")

for k, v in stats.items():
    print(k, v.shape)

data_mean torch.Size([17])
data_std torch.Size([17])
flux_mean torch.Size([])
flux_std torch.Size([])


In [4]:
stats

{'data_mean': tensor([ 9.9905e+04,  1.0110e+05, -2.1808e+05,  3.4544e+05,  7.1781e-01,
          7.1459e-01,  2.7186e+02,  2.7192e+02,  2.3915e+02,  2.6271e+02,
          2.4146e-01,  8.8596e-01, -2.5360e+00, -3.9970e+00,  6.2696e+00,
          8.5077e+02,  5.1629e+04]),
 'data_std': tensor([2.4345e+03, 7.6800e+02, 1.0905e+05, 5.0245e+05, 1.4583e-01, 1.6231e-01,
         4.8396e+00, 4.4669e+00, 5.4922e+00, 3.9706e+00, 4.3646e+00, 6.7067e+00,
         4.2867e+00, 5.2042e+00, 3.1096e+00, 5.9784e+02, 1.0582e+03]),
 'flux_mean': tensor(911702.5625),
 'flux_std': tensor(1037960.1250)}

## Checking ERA5 Files

In [53]:
param1 = torch.load("/vol/bitbucket/bet20/neural-lam/data/era5_uk/static/parameter_mean_32.pt")
param2 = torch.load("/vol/bitbucket/bet20/neural-lam/data/era5_uk/static/parameter_mean_64.pt")

In [56]:
param1

tensor([ 2.0165e+05,  1.3330e+05,  1.0130e+05,  7.0351e+04,  5.4557e+04,
         3.0270e+04,  1.4183e+04,  1.1059e+03,  3.0181e-06,  3.9068e-06,
         3.6811e-05,  3.2273e-04,  7.1151e-04,  1.9590e-03,  3.8835e-03,
         6.2932e-03,  2.1575e+02,  2.1831e+02,  2.2008e+02,  2.4069e+02,
         2.5225e+02,  2.6771e+02,  2.7597e+02,  2.8352e+02,  8.8710e+00,
         1.3517e+01,  1.6234e+01,  1.2451e+01,  1.0400e+01,  7.2419e+00,
         5.0889e+00,  1.8382e+00, -8.0580e-01, -1.6731e-01,  5.4722e-01,
         1.6034e+00,  1.8559e+00,  2.0115e+00,  2.2037e+00,  1.6394e+00,
         3.0533e-04,  2.5111e-04, -4.0914e-03, -7.0206e-03, -7.7413e-03,
        -7.5574e-03, -4.8157e-03,  2.7199e-04])

In [57]:
param2

tensor([ 2.0165e+05,  1.3330e+05,  1.0130e+05,  7.0351e+04,  5.4557e+04,
         3.0270e+04,  1.4183e+04,  1.1059e+03,  3.0181e-06,  3.9068e-06,
         3.6811e-05,  3.2273e-04,  7.1151e-04,  1.9590e-03,  3.8835e-03,
         6.2932e-03,  2.1575e+02,  2.1831e+02,  2.2008e+02,  2.4069e+02,
         2.5225e+02,  2.6771e+02,  2.7597e+02,  2.8352e+02,  8.8710e+00,
         1.3517e+01,  1.6234e+01,  1.2451e+01,  1.0400e+01,  7.2419e+00,
         5.0889e+00,  1.8382e+00, -8.0580e-01, -1.6731e-01,  5.4722e-01,
         1.6034e+00,  1.8559e+00,  2.0115e+00,  2.2037e+00,  1.6394e+00,
         3.0533e-04,  2.5111e-04, -4.0914e-03, -7.0206e-03, -7.7413e-03,
        -7.5574e-03, -4.8157e-03,  2.7199e-04])

In [48]:
import neural_lam.utils

stats = neural_lam.utils.load_dataset_stats("era5_uk")

for k, v in stats.items():
    print(k, v.shape)
    
stats

data_mean torch.Size([48])
data_std torch.Size([48])
flux_mean torch.Size([1])
flux_std torch.Size([1])


{'data_mean': tensor([ 2.0165e+05,  1.3330e+05,  1.0130e+05,  7.0351e+04,  5.4557e+04,
          3.0270e+04,  1.4183e+04,  1.1059e+03,  3.0181e-06,  3.9068e-06,
          3.6811e-05,  3.2273e-04,  7.1151e-04,  1.9590e-03,  3.8835e-03,
          6.2932e-03,  2.1575e+02,  2.1831e+02,  2.2008e+02,  2.4069e+02,
          2.5225e+02,  2.6771e+02,  2.7597e+02,  2.8352e+02,  8.8710e+00,
          1.3517e+01,  1.6234e+01,  1.2451e+01,  1.0400e+01,  7.2419e+00,
          5.0889e+00,  1.8382e+00, -8.0580e-01, -1.6731e-01,  5.4722e-01,
          1.6034e+00,  1.8559e+00,  2.0115e+00,  2.2037e+00,  1.6394e+00,
          3.0533e-04,  2.5111e-04, -4.0914e-03, -7.0206e-03, -7.7413e-03,
         -7.5574e-03, -4.8157e-03,  2.7199e-04]),
 'data_std': tensor([3.3787e+03, 2.5270e+03, 2.5154e+03, 1.9997e+03, 1.6560e+03, 3.3821e+03,
         1.0073e+03, 9.5701e+02, 1.4693e-07, 1.5479e-06, 3.0733e-05, 2.7484e-04,
         5.8946e-04, 1.4523e-03, 1.8634e-03, 2.0277e-03, 5.9726e+00, 5.3841e+00,
         4.7269e

# ERA5 Normalisation

In [5]:
nc_files = glob.glob(f'{RAW_ERA5_PATH}/2022*.nc')
nc_files.sort()
nc_files = nc_files[:12]
nc_files

['/vol/bitbucket/bet20/dataset/era5/global_full/2022_01.nc',
 '/vol/bitbucket/bet20/dataset/era5/global_full/2022_02.nc',
 '/vol/bitbucket/bet20/dataset/era5/global_full/2022_03.nc',
 '/vol/bitbucket/bet20/dataset/era5/global_full/2022_04.nc',
 '/vol/bitbucket/bet20/dataset/era5/global_full/2022_05.nc',
 '/vol/bitbucket/bet20/dataset/era5/global_full/2022_06.nc',
 '/vol/bitbucket/bet20/dataset/era5/global_full/2022_07.nc',
 '/vol/bitbucket/bet20/dataset/era5/global_full/2022_08.nc',
 '/vol/bitbucket/bet20/dataset/era5/global_full/2022_09.nc',
 '/vol/bitbucket/bet20/dataset/era5/global_full/2022_10.nc',
 '/vol/bitbucket/bet20/dataset/era5/global_full/2022_11.nc',
 '/vol/bitbucket/bet20/dataset/era5/global_full/2022_12.nc']

## Testing xarray sum method

In [6]:
xarray_sum = np.array([0]) # (N_vars, N_levels)
np_sum = np.array([0]) # (N_vars, N_levels)
total_points = 0

In [7]:
i = 0

for i in range(len(nc_files[:3])):
    data = xr.open_dataset(nc_files[i])
    data = era5_data_proc.uk_subset(data)

    num_points = len(data['time']) * len(data["latitude"]) * len(data["longitude"])
    total_points = total_points + num_points

    sums = np.array([data[var].sum(['time', 'latitude', 'longitude']).values for var in data.data_vars])
    print(sums.shape)
    xarray_sum = xarray_sum + sums
    
    vals = data.to_array().values
    vals = np.transpose(vals, (0, 2, 1, 3, 4))
    print(vals.shape)
    np_sum = np_sum + np.sum(vals, axis=(2, 3, 4))

    print(nc_files[i])
    print(num_points)
    
np.isclose(xarray_sum, np_sum).all()

(6, 8)
(6, 8, 124, 65, 57)
/vol/bitbucket/bet20/dataset/era5/global_full/2022_01.nc
459420
(6, 8)
(6, 8, 112, 65, 57)
/vol/bitbucket/bet20/dataset/era5/global_full/2022_02.nc
414960
(6, 8)
(6, 8, 124, 65, 57)
/vol/bitbucket/bet20/dataset/era5/global_full/2022_03.nc
459420


True

## ERA5 Normalisation from Xarray

In [8]:
var_sums = np.array([0]) # (N_vars, N_levels)
total_points = 0

for i in range(len(nc_files)):
    data = xr.open_dataset(nc_files[i])
    data = era5_data_proc.uk_subset(data)

    num_points = len(data['time']) * len(data["latitude"]) * len(data["longitude"])
    total_points = total_points + num_points

    vals = data.to_array().values # (N_vars, N_times, N_levels, N_lats, N_lons)
    vals = np.transpose(vals, (0, 2, 1, 3, 4)) # (N_vars, N_levels, N_times, N_lats, N_lons)
    var_sums = var_sums + np.sum(vals, axis=(2, 3, 4)) # (N_vars, N_levels)
    
    print(nc_files[i])
    print(num_points)
    
var_means = var_sums / total_points

/vol/bitbucket/bet20/dataset/era5/global_full/2022_01.nc
459420
/vol/bitbucket/bet20/dataset/era5/global_full/2022_02.nc
414960
/vol/bitbucket/bet20/dataset/era5/global_full/2022_03.nc
459420
/vol/bitbucket/bet20/dataset/era5/global_full/2022_04.nc
444600
/vol/bitbucket/bet20/dataset/era5/global_full/2022_05.nc
459420
/vol/bitbucket/bet20/dataset/era5/global_full/2022_06.nc
444600
/vol/bitbucket/bet20/dataset/era5/global_full/2022_07.nc
459420
/vol/bitbucket/bet20/dataset/era5/global_full/2022_08.nc
459420
/vol/bitbucket/bet20/dataset/era5/global_full/2022_09.nc
444600
/vol/bitbucket/bet20/dataset/era5/global_full/2022_10.nc
459420
/vol/bitbucket/bet20/dataset/era5/global_full/2022_11.nc
444600
/vol/bitbucket/bet20/dataset/era5/global_full/2022_12.nc
459420


In [24]:
total_points

5409300

In [23]:
var_sums

array([[ 1.09015224e+12,  7.20641034e+11,  5.47670102e+11,
         3.80296571e+11,  2.94897002e+11,  1.64101978e+11,
         7.65836121e+10,  5.85896535e+09],
       [ 1.63359959e+01,  2.11206421e+01,  1.97379381e+02,
         1.73464944e+03,  3.83042161e+03,  1.05280708e+04,
         2.09490841e+04,  3.39356692e+04],
       [ 1.16613773e+09,  1.18031132e+09,  1.19003267e+09,
         1.30140368e+09,  1.36391536e+09,  1.44728626e+09,
         1.49239670e+09,  1.53337186e+09],
       [ 5.06569929e+07,  7.46620240e+07,  8.92589810e+07,
         6.85779214e+07,  5.73118468e+07,  4.00946581e+07,
         2.82847254e+07,  1.02825966e+07],
       [-3.63395765e+06, -1.94027327e+05,  3.80659153e+06,
         9.34574615e+06,  1.05767191e+07,  1.12408657e+07,
         1.21226413e+07,  9.01224060e+06],
       [ 1.48082852e+03,  7.19104635e+02, -2.41637899e+04,
        -4.21965911e+04, -4.66380283e+04, -4.53147993e+04,
        -2.97861556e+04,  3.68742678e+02]])

In [21]:
sums = torch.load("/vol/bitbucket/bet20/neural-lam/data/era5_uk/static/parameter_sum.pt")

In [26]:
sums / total_points

tensor([ 5.9293e+06,  3.9194e+06,  2.9788e+06,  2.0686e+06,  1.6042e+06,
         8.9007e+05,  4.1705e+05,  3.2518e+04,  8.8743e-05,  1.1488e-04,
         1.0824e-03,  9.4895e-03,  2.0921e-02,  5.7604e-02,  1.1419e-01,
         1.8505e-01,  6.3438e+03,  6.4193e+03,  6.4711e+03,  7.0772e+03,
         7.4172e+03,  7.8718e+03,  8.1145e+03,  8.3365e+03,  2.6084e+02,
         3.9746e+02,  4.7735e+02,  3.6611e+02,  3.0579e+02,  2.1294e+02,
         1.4964e+02,  5.4050e+01, -2.3694e+01, -4.9195e+00,  1.6091e+01,
         4.7146e+01,  5.4570e+01,  5.9147e+01,  6.4797e+01,  4.8204e+01,
         8.9780e-03,  7.3838e-03, -1.2030e-01, -2.0643e-01, -2.2763e-01,
        -2.2222e-01, -1.4160e-01,  7.9976e-03])

In [16]:
var_means_flat = var_means.reshape(-1)

In [17]:
var_means_flat

array([ 2.01532960e+05,  1.33222604e+05,  1.01246021e+05,  7.03042115e+04,
        5.45166662e+04,  3.03370082e+04,  1.41577676e+04,  1.08312820e+03,
        3.01998335e-06,  3.90450558e-06,  3.64888953e-05,  3.20679097e-04,
        7.08117800e-04,  1.94629080e-03,  3.87279021e-03,  6.27357869e-03,
        2.15580155e+02,  2.18200380e+02,  2.19997535e+02,  2.40586339e+02,
        2.52142673e+02,  2.67555184e+02,  2.75894608e+02,  2.83469554e+02,
        9.36479635e+00,  1.38025297e+01,  1.65010225e+01,  1.26777811e+01,
        1.05950579e+01,  7.41217128e+00,  5.22890677e+00,  1.90091076e+00,
       -6.71798135e-01, -3.58692116e-02,  7.03712409e-01,  1.72771822e+00,
        1.95528425e+00,  2.07806291e+00,  2.24107394e+00,  1.66606411e+00,
        2.73756035e-04,  1.32938575e-04, -4.46708260e-03, -7.80074891e-03,
       -8.62182321e-03, -8.37720209e-03, -5.50647138e-03,  6.81682801e-05])

In [33]:
stats["data_mean"]

tensor([ 2.0165e+05,  1.3330e+05,  1.0130e+05,  7.0351e+04,  5.4557e+04,
         3.0270e+04,  1.4183e+04,  1.1059e+03,  3.0181e-06,  3.9068e-06,
         3.6811e-05,  3.2273e-04,  7.1151e-04,  1.9590e-03,  3.8835e-03,
         6.2932e-03,  2.1575e+02,  2.1831e+02,  2.2008e+02,  2.4069e+02,
         2.5225e+02,  2.6771e+02,  2.7597e+02,  2.8352e+02,  8.8710e+00,
         1.3517e+01,  1.6234e+01,  1.2451e+01,  1.0400e+01,  7.2419e+00,
         5.0889e+00,  1.8382e+00, -8.0580e-01, -1.6731e-01,  5.4722e-01,
         1.6034e+00,  1.8559e+00,  2.0115e+00,  2.2037e+00,  1.6394e+00,
         3.0533e-04,  2.5111e-04, -4.0914e-03, -7.0206e-03, -7.7413e-03,
        -7.5574e-03, -4.8157e-03,  2.7199e-04])

In [30]:
(var_means.reshape(-1) - stats["data_mean"].numpy()) / var_means.reshape(-1) * 100

array([-5.78497981e-02, -5.49517264e-02, -5.81295177e-02, -6.67404324e-02,
       -7.36404897e-02,  2.19520870e-01, -1.80437461e-01, -2.10392325e+00,
        6.38323152e-02, -5.90654638e-02, -8.81713980e-01, -6.38511983e-01,
       -4.79285271e-01, -6.55473173e-01, -2.75568534e-01, -3.12530227e-01,
       -7.65058840e-02, -5.11741518e-02, -3.53004133e-02, -4.22951863e-02,
       -4.23602706e-02, -5.80539910e-02, -2.56345328e-02, -1.60478954e-02,
        5.27338579e+00,  2.06821929e+00,  1.61669667e+00,  1.78957325e+00,
        1.84405534e+00,  2.29692162e+00,  2.67706836e+00,  3.30052266e+00,
       -1.99460922e+01, -3.66431718e+02,  2.22381464e+01,  7.19700542e+00,
        5.08440630e+00,  3.20155668e+00,  1.66960261e+00,  1.60263596e+00,
       -1.15336896e+01, -8.88954073e+01,  8.40951681e+00,  1.00011321e+01,
        1.02129109e+01,  9.78610260e+00,  1.25452183e+01, -2.98996600e+02])

In [13]:
np.isclose(var_means.reshape(-1), stats["data_mean"]).all()

False

In [30]:
var_stds = np.array([0]) # (N_vars, N_levels)
for i in range(len(nc_files[:3])):
    data = xr.open_dataset(nc_files[i])
    data = era5_data_proc.uk_subset(data)

    vals = data.to_array().values # (N_vars, N_times, N_levels, N_lats, N_lons)
    vals = np.transpose(vals, (0, 2, 1, 3, 4)) # (N_vars, N_levels, N_times, N_lats, N_lons)
    var_squared_diff = (vals - var_means.reshape(*var_means.shape, 1, 1, 1)) ** 2
    var_stds = var_stds + np.sum(var_squared_diff, axis=(2, 3, 4)) # (N_vars, N_levels)
    
    print(nc_files[i])
    print(num_points)


/vol/bitbucket/bet20/dataset/era5/global_full/2022_01.nc
459420
/vol/bitbucket/bet20/dataset/era5/global_full/2022_02.nc
459420
/vol/bitbucket/bet20/dataset/era5/global_full/2022_03.nc
459420


In [ ]:
var_means

array([[ 1.98566816e+05,  1.31345204e+05,  9.99637114e+04,
         6.94915794e+04,  5.39460403e+04,  3.31474312e+04,
         1.41598198e+04,  1.26508139e+03],
       [ 2.98321743e-06,  3.56682817e-06,  1.98416057e-05,
         1.88308410e-04,  4.45394453e-04,  1.11086685e-03,
         2.72969379e-03,  4.69097381e-03],
       [ 2.11742015e+02,  2.14685919e+02,  2.16439728e+02,
         2.36899708e+02,  2.48358562e+02,  2.62084934e+02,
         2.72406160e+02,  2.79824492e+02],
       [ 2.52679674e+01,  1.99496947e+01,  1.96528285e+01,
         1.57890576e+01,  1.36483263e+01,  1.05181391e+01,
         7.96240956e+00,  3.91867843e+00],
       [-2.01051010e+00, -2.21252477e+00, -2.24673299e+00,
        -3.94239711e-01,  3.00932753e-01,  8.27003147e-01,
         1.69441152e+00,  2.29910446e+00],
       [ 2.05483847e-03,  3.22094799e-03,  4.52772609e-04,
         8.94847508e-04, -2.17357494e-03, -6.33947440e-03,
        -5.22065071e-03, -1.16850669e-03]])

## ERA5 Normalisation with dataloader

In [ ]:
import glob
import os
import torch
from tqdm import tqdm
from neural_lam.era5_dataset import ERA5UKDataset
from neural_lam import utils

dataset = "era5_uk_full"
batch_size = 2
n_workers = 4

In [ ]:
static = utils.load_static_data(dataset)

In [ ]:
static["data_std"]

tensor([2.5464e+03, 2.0264e+03, 2.2975e+03, 2.0187e+03, 1.7620e+03, 5.8213e+03,
        1.2267e+03, 1.1808e+03, 1.6962e-07, 1.1838e-06, 1.2919e-05, 1.5098e-04,
        3.6187e-04, 9.0703e-04, 1.3290e-03, 1.1877e-03, 6.1122e+00, 5.6558e+00,
        3.5887e+00, 4.8513e+00, 5.2299e+00, 5.2967e+00, 4.2371e+00, 2.8118e+00,
        1.4652e+01, 1.2807e+01, 1.9964e+01, 1.7005e+01, 1.4406e+01, 1.1302e+01,
        9.8326e+00, 7.3897e+00, 1.1996e+01, 1.2976e+01, 2.1520e+01, 1.8191e+01,
        1.5043e+01, 1.1692e+01, 9.7814e+00, 7.3720e+00, 3.0445e-02, 6.3717e-02,
        1.3932e-01, 2.6964e-01, 3.2182e-01, 3.4959e-01, 3.4065e-01, 1.1852e-01])

In [ ]:
ds = ERA5UKDataset(
    dataset,
    split="train",
    standardize=False,
)  # Without standardization

In [ ]:
loader = torch.utils.data.DataLoader(
    ds, batch_size, shuffle=False, num_workers=n_workers
)

In [ ]:
means = []
squares = []
flux_means = []
flux_squares = []
i = 0
LIMIT = (31 * 4 + 28 * 4 + 31 * 4) // batch_size
for init_batch, target_batch, forcing_batch in tqdm(loader):
    if i == LIMIT:
        break
    i += 1
    batch = torch.cat(
        (init_batch, target_batch), dim=1
    )  # (N_batch, N_t, N_grid, d_features)
    means.append(torch.mean(batch, dim=(1, 2)))  # (N_batch, d_features,)
    squares.append(
        torch.mean(batch**2, dim=(1, 2))
    )  # (N_batch, d_features,)


mean = torch.mean(torch.cat(means, dim=0), dim=0)  # (d_features)

 25%|██▌       | 180/716 [00:10<00:32, 16.52it/s]


# Test MEPS dataset

In [ ]:
import glob
import os
import torch
from neural_lam.weather_dataset import WeatherDataset

dataset_name = "meps_example"
batch_size = 2
n_workers = 4

train_set = WeatherDataset(
    dataset_name,
    split="train",
)

train_loader = torch.utils.data.DataLoader(
    train_set,
    batch_size,
    shuffle=True,
    num_workers=n_workers,
)

In [ ]:
print(len(train_loader))
dataiter = iter(train_loader)
init_states, target_states, forcing = next(dataiter)

2


In [ ]:
print(init_states.shape)
print(target_states.shape)
print(forcing.shape)

torch.Size([1, 3705, 48])
torch.Size([2, 19, 63784, 17])
torch.Size([6, 3705, 12])


# Test ERA5 dataset

## Train Split

In [ ]:
import glob
import os
import torch
from neural_lam.era5_dataset import ERA5UKDataset

dataset_name = "era5_uk_full"
batch_size = 2
n_workers = 4


In [ ]:
train_set = ERA5UKDataset(
    dataset_name,
    pred_length=1,
    split="train",
    standardize=False,
)
train_loader = torch.utils.data.DataLoader(
    train_set,
    batch_size,
    shuffle=True,
    num_workers=n_workers,
)

print(len(train_loader))
dataiter = iter(train_loader)
init_states, target_states, forcing = next(dataiter)
init_states

729


tensor([[[[ 1.9736e+05,  1.3203e+05,  1.0197e+05,  ..., -2.6944e-01,
           -2.1301e-01, -1.2030e-02],
          [ 1.9744e+05,  1.3210e+05,  1.0204e+05,  ..., -2.9039e-01,
           -1.7324e-01, -3.4319e-03],
          [ 1.9754e+05,  1.3218e+05,  1.0212e+05,  ..., -2.5708e-01,
           -6.5231e-02,  1.8601e-02],
          ...,
          [ 2.0049e+05,  1.3320e+05,  1.0142e+05,  ...,  4.6545e-02,
            4.2540e-01, -9.6399e-02],
          [ 2.0052e+05,  1.3320e+05,  1.0142e+05,  ...,  1.3091e-01,
            2.2012e-01, -3.8899e-02],
          [ 2.0055e+05,  1.3319e+05,  1.0142e+05,  ...,  1.5402e-01,
           -2.2430e-01,  9.8133e-02]],

         [[ 1.9758e+05,  1.3228e+05,  1.0215e+05,  ..., -1.1360e-01,
            1.1694e-01, -1.1252e-01],
          [ 1.9765e+05,  1.3235e+05,  1.0221e+05,  ..., -9.8804e-03,
            1.9138e-02, -1.3778e-01],
          [ 1.9772e+05,  1.3242e+05,  1.0227e+05,  ...,  7.9325e-02,
           -6.4693e-02, -1.1628e-01],
          ...,
     

In [ ]:
train_set = ERA5UKDataset(
    dataset_name,
    pred_length=1,
    split="train",
    standardize=True,
)
train_loader = torch.utils.data.DataLoader(
    train_set,
    batch_size,
    shuffle=True,
    num_workers=n_workers,
)

print(len(train_loader))
dataiter = iter(train_loader)
init_states, target_states, forcing = next(dataiter)
init_states

729


tensor([[[[ 2.9441,  2.0760,  1.3319,  ...,  0.7017,  0.3570, -0.1295],
          [ 2.9493,  2.1082,  1.3931,  ...,  0.7084,  0.3536, -0.0852],
          [ 2.9531,  2.1405,  1.4528,  ...,  0.6300,  0.3673, -0.0114],
          ...,
          [ 2.9980,  3.2260,  2.6165,  ...,  0.7734,  0.6669, -0.6510],
          [ 2.9916,  3.2405,  2.6165,  ...,  1.1254,  0.8740, -0.1344],
          [ 2.9865,  3.2550,  2.6151,  ...,  1.1254,  0.8997,  1.0021]],

         [[ 2.9018,  2.1098,  1.4884,  ..., -2.6577,  0.3793, -0.2131],
          [ 2.9095,  2.1502,  1.5538,  ..., -3.5968,  0.0506, -0.4788],
          [ 2.9159,  2.1921,  1.6164,  ..., -3.8821, -0.5536, -0.5477],
          ...,
          [ 2.9005,  3.1873,  2.5810,  ..., -0.1290,  0.9322, -0.0507],
          [ 2.8941,  3.1970,  2.5781,  ...,  0.0828,  1.2078,  0.2100],
          [ 2.8877,  3.2099,  2.5753,  ...,  0.1179,  1.2232,  1.0464]]],


        [[[ 3.1045,  1.8560,  0.5640,  ..., -0.2758, -0.7146, -0.1385],
          [ 3.1032,  1.8738,

In [ ]:
path = "/vol/bitbucket/bet20/neural-lam/data/era5_uk_full/samples/train"
files = glob.glob(os.path.join(path, "*.npy"))
len(files)

1460

In [ ]:
print(init_states.shape)
print(target_states.shape)
print(forcing.shape)

torch.Size([2, 2, 3705, 48])
torch.Size([2, 1, 3705, 48])
torch.Size([2, 1, 3705, 0])


In [ ]:
val_set = ERA5UKDataset(
    dataset_name,
    pred_length=28,
    split="val",
)

val_loader = torch.utils.data.DataLoader(
    val_set,
    batch_size,
    shuffle=True,
    num_workers=n_workers,
)

valiter = iter(val_loader)
init_states, target_states, forcing = next(valiter)

print(len(val_set))
print(init_states.shape)
print(target_states.shape)
print(forcing.shape)

376
torch.Size([2, 2, 3705, 48])
torch.Size([2, 28, 3705, 48])
torch.Size([2, 28, 3705, 0])


## Train Val Split

In [ ]:
import os
import glob
import datetime

In [ ]:
# load time step data
sample_dir_path = "data/era5_uk/samples/train"
sample_files = glob.glob(f'{sample_dir_path}/*.npy')
sample_files.sort()
sample_files = [os.path.basename(f)[:-4] for f in sample_files]
sample_times = [datetime.datetime.strptime(f, '%Y%m%d%H%M%S') for f in sample_files]

In [ ]:
sample_files[-1]

'20221231180000'

In [ ]:
for i in range(1, len(sample_times)):
    delta = sample_times[i] - sample_times[i-1]
    if delta != datetime.timedelta(hours=6):
        print(f"Missing time step: {sample_times[i-1]} -> {sample_times[i]} ({delta})")

In [ ]:
sample_dir_path = "data/era5_uk/samples/val"
# directories = [name for name in os.listdir(sample_dir_path)]
# directories.sort()
# directories
val_months = ["01", "04", "07", "10"]

In [ ]:
month_path = os.path.join(sample_dir_path, val_months[1])
print(month_path)
sample_files = glob.glob(f'{month_path}/*.npy')
sample_files.sort()
sample_files

data/era5_uk_full/samples/val/04


['data/era5_uk_full/samples/val/04/20230401000000.npy',
 'data/era5_uk_full/samples/val/04/20230401060000.npy',
 'data/era5_uk_full/samples/val/04/20230401120000.npy',
 'data/era5_uk_full/samples/val/04/20230401180000.npy',
 'data/era5_uk_full/samples/val/04/20230402000000.npy',
 'data/era5_uk_full/samples/val/04/20230402060000.npy',
 'data/era5_uk_full/samples/val/04/20230402120000.npy',
 'data/era5_uk_full/samples/val/04/20230402180000.npy',
 'data/era5_uk_full/samples/val/04/20230403000000.npy',
 'data/era5_uk_full/samples/val/04/20230403060000.npy',
 'data/era5_uk_full/samples/val/04/20230403120000.npy',
 'data/era5_uk_full/samples/val/04/20230403180000.npy',
 'data/era5_uk_full/samples/val/04/20230404000000.npy',
 'data/era5_uk_full/samples/val/04/20230404060000.npy',
 'data/era5_uk_full/samples/val/04/20230404120000.npy',
 'data/era5_uk_full/samples/val/04/20230404180000.npy',
 'data/era5_uk_full/samples/val/04/20230405000000.npy',
 'data/era5_uk_full/samples/val/04/2023040506000

In [ ]:
import shutil
import glob
import os

# Specify the source and destination directories
src_dir = 'data/era5_uk_full/samples/train/'
dst_dir = 'data/era5_uk_full/samples/val/01'

os.makedirs(dst_dir, exist_ok=True)

# Use glob to get all .npy files that start with 2023 in the source directory
files = glob.glob(os.path.join(src_dir, '202301*.npy'))

# print(files)
# print(len(files))

# Move each file to the destination directory
for file in files:
    shutil.move(file, dst_dir)

In [ ]:
31 * 4

124

In [ ]:

pred_length = 28
sample_length = pred_length + 2
val_samples = []
sample_dir_path = "./data/era5_uk_full/samples/val"
month_samples = {}

for month in val_months:
    month_dir = os.path.join(sample_dir_path, month)
    data_files = glob.glob(os.path.join(month_dir, "*.npy"))
    data_files.sort()
    month_samples[month] = data_files
    n_samples = len(data_files) - sample_length + 1
    for i in range(n_samples):
        val_samples.append((month, i))
    

N = len(val_samples)
N

376

In [ ]:
idx = 332
month, idx = val_samples[idx]
sample_names = month_samples[month]

sample_names[idx], idx

('./data/era5_uk_full/samples/val/10/20231013180000.npy', 51)

In [ ]:
# verify number of time steps
val_files = glob.glob(f'{src_dir}/10/*.npy')
len(val_files)

124

In [ ]:
val_files = glob.glob(f'{src_dir}/*.npy')
# val_files.sort()
# wanted = [v for v in val_files if "202301" in v or "202310" in v or "202304" in v or "202307" in v]
# wanted

[]